# Roberta fine-tunning in mnli (original + negative sentences)

The code is from https://github.com/facebookresearch/fairseq/blob/main/examples/roberta/README.md



## preparation of environment

https://github.com/facebookresearch/fairseq/issues/3723 解决non-iterable的问题 

This is fixed in the master branch of hydra with the following commit but no release was made afterwards:
facebookresearch/hydra@8fa67de

It is also fixed by downgrading to hydra-core==1.0.7

In [2]:
!nvidia-smi

Fri Sep  2 13:27:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/memoire"

/content/drive/MyDrive/memoire


In [5]:
%%bash
pip install regex requests hydra-core omegaconf
pip install hydra-core==1.0.7
pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=20901def7b1ae6f26e596de3ae0be30821fcd06721003900896457af5d884b93
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=e74aadf879f1cef71bf80909d56fb6f2bd346784a929e689f79e3adbad296544
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.2.3
    Uninstalling omegaconf-2.2.3:
      Successfully unin

In [6]:
import torch
from fairseq.data.data_utils import collate_tokens

In [7]:
%%bash
pip install transformers
pip install datasets
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

## evaluation on a whole file (dev)

In [ ]:
# ROBERTA LARGE SNLI
checkpoint = "textattack/roberta-base-MNLI"
tokenizer_mnli = AutoTokenizer.from_pretrained(checkpoint)
model_mnli = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of the model checkpoint at textattack/roberta-base-MNLI were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### dev_matched 

In [ ]:
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_mnli.cuda() # gpu
model_mnli.eval()
with open('corpus/MNLI/dev_matched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer_mnli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_mnli(**inputs.to(model_mnli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# | Accuracy:  0.8809984717269486

| Accuracy:  0.8809984717269486


### dev_mismatched

In [ ]:
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_mnli.cuda() # gpu
model_mnli.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer_mnli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_mnli(**inputs.to(model_mnli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# | Accuracy:  0.8783563873067535

| Accuracy:  0.8783563873067535


### neg mnli

In [ ]:
# with roberta_mnli with open('data/mnli_completed_with_importance_neg_only.tsv') as fin:
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_mnli.cuda() # gpu
model_mnli.eval()
with open('data/mnli_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer_mnli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_mnli(**inputs.to(model_mnli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# Accuracy:  0.6793333333333333

| Accuracy:  0.6793333333333333


## fine-tunning

### datasets

In [ ]:
data_files = './data/mnli_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [ ]:
label_map = {'contradiction':0 , 'neutral':1 , 'entailment':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns(['label', 'index'])
neg_datasets = neg_datasets.rename_column('Unnamed: 0', 'index')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [ ]:
# # download the checkpoint
# checkpoint = "textattack/roberta-base-MNLI"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
def tokenize_function(example):
    return tokenizer_mnli(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_mnli)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "roberta-base-mnli-fine-tunned-neg-snli", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mnli")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_mnli,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_mnli,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: importance, text, hypothesis, index. If importance, text, hypothesis, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.614050,0.780952
2,No log,0.586291,0.819048
3,No log,0.603724,0.809524


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: importance, text, hypothesis, index. If importance, text, hypothesis, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 32


Saving model checkpoint to roberta-base-mnli-fine-tunned-neg-snli/checkpoint-30
Configuration saved in roberta-base-mnli-fine-tunned-neg-snli/checkpoint-30/config.json
Model weights saved in roberta-base-mnli-fine-tunned-neg-snli/checkpoint-30/pytorch_model.bin
tokenizer config file saved in roberta-base-mnli-fine-tunned-neg-snli/checkpoint-30/tokenizer_config.json
Special tokens file saved in roberta-base-mnli-fine-tunned-neg-snli/checkpoint-30/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: importance, text, hypothesis, index. If importance, text, hypothesis, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 32
Saving model checkpoint to roberta-base-mnli-fine-tunned-neg-snli/checkpoint-60
Configuration saved in roberta-base-mnli-fine-tunn

TrainOutput(global_step=90, training_loss=0.48443400065104164, metrics={'train_runtime': 57.6802, 'train_samples_per_second': 49.15, 'train_steps_per_second': 1.56, 'total_flos': 86324840110908.0, 'train_loss': 0.48443400065104164, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [9]:
# download the checkpoint
checkpoint = "roberta-base-mnli-fine-tunned-neg-snli/checkpoint-90"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [10]:
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8478437754271766


### test on important and unimportante sentences

In [ ]:
test_neg_datasets = neg_datasets['test']
test_neg_datasets

Dataset({
    features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets_importance

Dataset({
    features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 336
})

In [ ]:
test_neg_datasets_unimportance

Dataset({
    features: ['index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 80
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.8044444444444444

| Accuracy:  0.8044444444444444


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.7767857142857143

| Accuracy:  0.7767857142857143


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.85

| Accuracy:  0.85


# Roberta fine-tunning in snli (original + negative sentences)

## preparation of environment

https://github.com/facebookresearch/fairseq/issues/3723 解决non-iterable的问题 

This is fixed in the master branch of hydra with the following commit but no release was made afterwards:
facebookresearch/hydra@8fa67de

It is also fixed by downgrading to hydra-core==1.0.7

In [ ]:
!nvidia-smi

Sun Aug 14 07:44:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/memoire"

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install regex requests hydra-core omegaconf
pip install hydra-core==1.0.7
pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=371bbff883232b82e4bd0bd2adf73d0d50715f8a4a6140d80ed8b600d1f2e084
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=9c4b8ba2da41eaa2fda065899e054f3892b345a688c7967ac8c5c821f8e2a2c0
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3

In [ ]:
import torch
from fairseq.data.data_utils import collate_tokens

In [ ]:
%%bash
pip install transformers
pip install datasets
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

## load the model roberta.large and test on one sentence

In [ ]:
# ROBERTA LARGE SNLI
checkpoint = "boychaboy/SNLI_roberta-base"
tokenizer_snli = AutoTokenizer.from_pretrained(checkpoint)
model_snli = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

## evaluation on a whole file (dev)

### original 

In [ ]:
# with roberta large snli

label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_snli.cuda() # gpu
model_snli.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer_snli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_snli(**inputs.to(model_snli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# Accuracy:  

| Accuracy:  0.9022


### mnli_completed_with_importance

In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_snli.cuda() # gpu
model_snli.eval()
with open('data/snli_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer_snli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_snli(**inputs.to(model_snli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# Accuracy: 0.5246666666666666

| Accuracy:  0.5246666666666666


## fine-tunning on negative sentences

### load datasets

In [ ]:
data_files = './data/snli_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-95c6bc427456eefc/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [ ]:
label_map = {'entailment':0 , 'neutral':1 , 'contradiction':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [ ]:
# # ROBERTA LARGE SNLI
# checkpoint = "boychaboy/SNLI_roberta-large"
# tokenizer_snli = AutoTokenizer.from_pretrained(checkpoint)
# model_snli = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
def tokenize_function(example):
    return tokenizer_snli(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_snli)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "roberta-base-snli-fine-tunned-neg-snli", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", 'mnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_snli,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_snli,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
# 8 min 17s

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, Unnamed: 0, hypothesis, importance, index. If text, Unnamed: 0, hypothesis, importance, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.752795,0.800000
2,No log,0.823095,0.800000
3,No log,0.828556,0.809524


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, Unnamed: 0, hypothesis, importance, index. If text, Unnamed: 0, hypothesis, importance, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 32
Saving model checkpoint to roberta-base-snli-fine-tunned-neg-snli/checkpoint-30
Configuration saved in roberta-base-snli-fine-tunned-neg-snli/checkpoint-30/config.json
Model weights saved in roberta-base-snli-fine-tunned-neg-snli/checkpoint-30/pytorch_model.bin
tokenizer config file saved in roberta-base-snli-fine-tunned-neg-snli/checkpoint-30/tokenizer_config.json
Special tokens file saved in roberta-base-snli-fine-tunned-neg-snli/checkpoint-30/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClas

TrainOutput(global_step=90, training_loss=0.16906743579440647, metrics={'train_runtime': 51.3134, 'train_samples_per_second': 55.249, 'train_steps_per_second': 1.754, 'total_flos': 61274078833590.0, 'train_loss': 0.16906743579440647, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "roberta-base-snli-fine-tunned-neg-snli/checkpoint-90"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Didn't find file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/added_tokens.json. We won't load it.
loading file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/vocab.json
loading file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/merges.txt
loading file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/tokenizer.json
loading file None
loading file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/special_tokens_map.json
loading file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/tokenizer_config.json
loading configuration file roberta-base-snli-fine-tunned-neg-snli/checkpoint-90/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base-snli-fine-tunned-neg-snli/checkpoint-90",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "finetuning_task": "snli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

In [ ]:
# with roberta (not fine-tunned on mnli)
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8032


### test on important and unimportante sentences

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.8466666666666667


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.840625


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.8526315789473684


# Roberta fine-tunning in rte (original + negative sentences)

## preparation of environment

https://github.com/facebookresearch/fairseq/issues/3723 解决non-iterable的问题 

This is fixed in the master branch of hydra with the following commit but no release was made afterwards:
facebookresearch/hydra@8fa67de

It is also fixed by downgrading to hydra-core==1.0.7

In [ ]:
!nvidia-smi

Sun Aug 14 09:33:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/memoire"

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install regex requests hydra-core omegaconf
pip install hydra-core==1.0.7
pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=3ef1ec15019ad17b94111b5ceb37513839941ceb6e40a3789b7f007b6ed2be43
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=b095a6c716e4dcf4f88cf28992e0c2d8338cd040a68a7b71d95fd940f54ebfd8
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3

In [ ]:
import torch
from fairseq.data.data_utils import collate_tokens

In [ ]:
%%bash
pip install transformers
pip install datasets
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

## load the model roberta.large and test on one sentence

choose 1

In [ ]:
# ROBERTA LARGE RTE

checkpoint = 'textattack/roberta-base-RTE'
tokenizer_rte = AutoTokenizer.from_pretrained(checkpoint)
model_rte = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

## evaluation on a whole file (dev)

### test the label's order

In [ ]:
raw_inputs = [
    "I like you",
    "I love you",
]
inputs = tokenizer_rte(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  0, 100, 101,  47,   2,   2, 100, 657,  47,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
outputs = model_rte(**inputs.to(model_rte.device))

In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.6716,  0.8936]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
print(outputs.logits.shape)

torch.Size([1, 2])


In [ ]:
label_map = model_rte.config.id2label
label_map

{0: 'entailment', 1: 'not_entailment'}

In [ ]:
label_map[0]='entailment'
label_map[1]='not_entailment'


In [ ]:
label_map

{0: 'entailment', 1: 'not_entailment'}

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

not_entailment


### dev

In [ ]:
# evaluate on dev
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_rte.cuda() # gpu
model_rte.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_rte(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_rte(**inputs.to(model_rte.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7833935018050542


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_rte.cuda() # gpu
model_rte.eval()
with open('data/rte_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer_rte(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_rte(**inputs.to(model_rte.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7413333333333333


## Fine-tunning in a negative sentences dataset

### Download the datasets

In [ ]:
data_files = './data/rte_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2da8e5812fadb932/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [ ]:
label_map = {'not_entailment':1, 'entailment':0}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [ ]:
neg_datasets[1]

{'Unnamed: 0': 1,
 'index': 2,
 'text': "The most surprising news story of the past week must be the 'wonderful' story that the AIDS epidemic in India is under control.",
 'hypothesis': 'Polio is not under control in the world.',
 'importance': '0',
 'labels': 1}

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [ ]:
# # download the checkpoint
# checkpoint = "howey/roberta-large-rte"
# tokenizer_rte = AutoTokenizer.from_pretrained(checkpoint)
# model_rte = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
def tokenize_function(example):
    return tokenizer_rte(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_rte)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "roberta-base-rte-fine-tunned-neg-rte", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "rte")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_rte,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_rte,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, Unnamed: 0, hypothesis, importance, index. If text, Unnamed: 0, hypothesis, importance, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.419612,0.847619
2,No log,0.420549,0.885714
3,No log,0.497067,0.876190
4,No log,0.530596,0.885714
5,No log,0.601288,0.876190
6,No log,0.770259,0.857143
7,No log,0.793115,0.866667
8,No log,0.740744,0.876190
9,0.206500,0.835000,0.876190
10,0.206500,0.818536,0.876190


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, Unnamed: 0, hypothesis, importance, index. If text, Unnamed: 0, hypothesis, importance, index are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 16
Saving model checkpoint to roberta-base-rte-fine-tunned-neg-rte/checkpoint-60
Configuration saved in roberta-base-rte-fine-tunned-neg-rte/checkpoint-60/config.json
Model weights saved in roberta-base-rte-fine-tunned-neg-rte/checkpoint-60/pytorch_model.bin
tokenizer config file saved in roberta-base-rte-fine-tunned-neg-rte/checkpoint-60/tokenizer_config.json
Special tokens file saved in roberta-base-rte-fine-tunned-neg-rte/checkpoint-60/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification

TrainOutput(global_step=600, training_loss=0.17798798322677611, metrics={'train_runtime': 253.6239, 'train_samples_per_second': 37.26, 'train_steps_per_second': 2.366, 'total_flos': 320841320905200.0, 'train_loss': 0.17798798322677611, 'epoch': 10.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "roberta-base-rte-fine-tunned-neg-rte/checkpoint-600"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Didn't find file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/added_tokens.json. We won't load it.
loading file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/vocab.json
loading file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/merges.txt
loading file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/tokenizer.json
loading file None
loading file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/special_tokens_map.json
loading file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/tokenizer_config.json
loading configuration file roberta-base-rte-fine-tunned-neg-rte/checkpoint-600/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base-rte-fine-tunned-neg-rte/checkpoint-600",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "finetuning_task": "glue:rte",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hid

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7256317689530686


### test on important and unimportante sentences

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8577777777777778


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8333333333333334


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.9433962264150944
